In [1]:
import pathlib
import sys 
import random
import json
import cv2 
import base64
import requests as req
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from io import BytesIO  
%matplotlib inline
import sklearn.metrics.pairwise as pairwise

#Importing ActionClassificationCosine

In [2]:
def cosineDistanceMatching(poseVector1,poseVector2):
    '''
    Returns the cosine similarity as a distance function between the two L2 normalized vectors.
    The distance is inversely porportional to the similarity between the two vectors
    '''
    poseVector1 , poseVector2 = np.array(poseVector1).reshape(1,-1) , np.array(poseVector2).reshape(1,-1)
    cosineDistance = pairwise.cosine_distances(poseVector1 , poseVector2) 
    distance = np.sqrt(cosineDistance * 2)
    return distance

In [3]:
import import_ipynb
import ActionClassificationCosine 

importing Jupyter notebook from ActionClassificationCosine.ipynb
importing Jupyter notebook from ActionClassificationCosineDatasetGen.ipynb
Loading Poses Data....
Initializing VPTREE....


#Setting constants

In [4]:
#constants
PART_NAMES = [
    "nose", "leftEye", "rightEye", "leftEar", "rightEar", "leftShoulder",
    "rightShoulder", "leftElbow", "rightElbow", "leftWrist", "rightWrist",
    "leftHip", "rightHip", "leftKnee", "rightKnee", "leftAnkle", "rightAnkle"
]
PART_IDS = {pn: pid for pid, pn in enumerate(PART_NAMES)} ;PART_IDS
CONNECTED_PART_NAMES = [
    ("leftHip", "leftShoulder"), ("leftElbow", "leftShoulder"),
    ("leftElbow", "leftWrist"), ("leftHip", "leftKnee"),
    ("leftKnee", "leftAnkle"), ("rightHip", "rightShoulder"),
    ("rightElbow", "rightShoulder"), ("rightElbow", "rightWrist"),
    ("rightHip", "rightKnee"), ("rightKnee", "rightAnkle"),
    ("leftShoulder", "rightShoulder"), ("leftHip", "rightHip")
]

CONNECTED_PART_INDICES = [(PART_IDS[a], PART_IDS[b]) for a, b in CONNECTED_PART_NAMES];


#Helper functions

In [5]:
def getBase64Image(filePath):
    with open(filePath, "rb") as img_file:
        my_string = base64.b64encode(img_file.read())
        my_string = my_string.decode('utf-8')
    return my_string

In [6]:
def convertToBase64(image):
    retval, buffer = cv2.imencode('.jpg', image)
    my_string = base64.b64encode(buffer)
    my_string = my_string.decode('utf-8')
    return my_string

In [7]:
def get_adjacent_keypoints(keypoints):
    '''
    Helper function of draw_skel_and _kp
    Returns 2 coord of 2 points where line needs to be drawn
    EXAMPLE: [[X1,Y1],[X2,Y2]]
    '''
    results = []
    for left, right in CONNECTED_PART_INDICES:
       results.append(
           np.array([
                [ keypoints[left]['position']['x'] , keypoints[left]['position']['y'] ],
                [ keypoints[right]['position']['x'] , keypoints[right]['position']['y'] ]
                    ]
           ).astype(np.int32)
        )
    return results
            

In [8]:
#Sending data to server
def getPoses(image_string):
    url2 = 'http://localhost:3000/postImage'
    data = {'imgBase64':'data:image/png;base64,'+image_string}
    r = req.post(url=url2 , data = data)
    poses = r.json()
    return poses

In [9]:
def draw_keypoints(poses,img):
    cv_keypoints= []
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        for keypoint in keypoints:
            x,y,score = round(keypoint['position']['x']) ,round(keypoint['position']['y']),keypoint['score']
            cv_keypoints.append(cv2.KeyPoint(x,y , 10. *score))
    out_img = cv2.drawKeypoints(img, cv_keypoints , outImage=np.array([]) ) 

In [10]:
def draw_skelton(poses,img):
    '''
    This function is not used anywhere. 
    Written to demonstrate how skelton can be drawn
    '''
    adjacent_keypoints = []
    out_img = img
    for pose in poses['detectionList']:
        keypoints  = pose['keypoints']
        new_keypoints = get_adjacent_keypoints(keypoints)
        adjacent_keypoints.extend(new_keypoints)
    out_img = cv2.polylines(out_img , adjacent_keypoints, isClosed=False, color=(255,255,0))
    return out_img    

In [11]:
def draw_skel_and_kp(poses , img, color,classes):
    out_img = img
    adjacent_keypoints = []
    cv_keypoints = []
    #For every pose of the player
    for i,pose in enumerate(poses['detectionList']):
        keypoints = pose['keypoints']
        new_keypoint = get_adjacent_keypoints(keypoints)
        adjacent_keypoints.extend(new_keypoint)
        
        for keypoint in keypoints:
            x,y,score = round(keypoint['position']['x']) ,round(keypoint['position']['y']),keypoint['score']
            cv_keypoints.append(cv2.KeyPoint(x,y , 10. * score))
        cv2.putText(out_img, classes[i][0], (x,y), cv2.FONT_ITALIC, 0.6, (0, 255, 0), 1)
    
    out_img = cv2.drawKeypoints(
        img, cv_keypoints , outImage=np.array([]), color=color,
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS )
    out_img = cv2.polylines(out_img , adjacent_keypoints , isClosed=False, color=list(color))
    return out_img

#Sending to server

In [ ]:
#Loading model
r = req.get(url='http://localhost:3000/loadModel')

In [18]:

img = cv2.imread('/Users/sandeep/Desktop/dataandmodles/data/pz3Pointer.png')
myString =convertToBase64(img)
poses = getPoses(myString)
#out_img = draw_skel_and_kp(poses,img,[255,0,0])

#out_img = draw_visibilty_cone(poses,img)
#cv2.imwrite('/Users/sandeep/Desktop/dataandmodles/data/croppedPose.JPG' ,out_img) 

In [19]:
classes = ActionClassificationCosine.fit(poses)

finding match
Searching Tree..
Finished Searching Tree..
Found match
finding match
Searching Tree..
Finished Searching Tree..
Found match
finding match
Searching Tree..
Finished Searching Tree..
Found match
finding match
Searching Tree..
Finished Searching Tree..
Found match
finding match
Searching Tree..
Finished Searching Tree..
Found match
finding match
Searching Tree..
Finished Searching Tree..
Found match
finding match
Searching Tree..
Finished Searching Tree..
Found match
finding match
Searching Tree..
Finished Searching Tree..
Found match
finding match
Searching Tree..
Finished Searching Tree..
Found match
finding match
Searching Tree..
Finished Searching Tree..
Found match


In [20]:
len(classes)

10

In [21]:
out_img = draw_skel_and_kp(poses,img,[255,0,0],classes)
cv2.imwrite('/Users/sandeep/Desktop/dataandmodles/data/cosineResult2.JPG' ,out_img) 

True

In [ ]:
%debug

In [ ]:
videoPath ='/Users/sandeep/Desktop/dataandmodles/data/shoot.mov'
cap = cv2.VideoCapture(videoPath)
count = 0
while cap.isOpened():
    ret,frame = cap.read()
    if ret:
        #do stuff
        myString =convertToBase64(frame)
        poses = getPoses(myString)
        #frame = draw_visibilty_cone(poses,frame)
        frame = draw_skel_and_kp(poses,frame,[255,0,0])
#         End of do stuff
        cv2.imshow('test', frame)
        #cv2.imwrite(f"/Users/sandeep/Desktop/dataandmodles/data/coneVision/{count}.png" , frame)
        count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break  
    else:
        cap.release()
        break
cap.release()
cv2.destroyAllWindows()
